In [1]:
import pandas as pd, numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
import pickle

sns.set_palette("deep", desat=0.6)
sns.set_context(rc={"figure.figsize": (8,4)})
import json
from urllib.request import urlopen
import requests
from shapely.geometry import shape, Point

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from datetime import datetime as dt
import time
import folium
from IPython.core.display import HTML

# Utility Functions

In [5]:
def getcoords(dframe):
    dfgeo1 = dframe.dropna(subset = ['LATITUDE','LONGITUDE'])
    coords = dfgeo1.as_matrix(columns=['LATITUDE','LONGITUDE'])
    return coords

In [6]:
def applyDBSCAN(coordinates, epsilon, min_samples):
    start_time = time.time()
    db = DBSCAN(eps=epsilon, min_samples = min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coordinates))
    cluster_labels = db.labels_
    # get the number of clusters
    num_clusters = len(set(cluster_labels))
    print(num_clusters)
    clusters = pd.Series([coordinates[cluster_labels==n] for n in range(num_clusters)])
    message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
    print(message.format(len(coordinates), num_clusters, 100*(1 - float(num_clusters) / len(coordinates)), time.time()-start_time))
    return clusters

In [7]:
def get_centermost_point(clusters):
    centroid = (MultiPoint(clusters).centroid.x, MultiPoint(clusters).centroid.y)
    centermost_point = min(clusters, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

In [8]:
#Find the point in each cluster that is closest to its centroid
def gethotspots(clusters, df, min_sample):
    centermost_points = []
    for cluster in clusters.iteritems():
        if len(cluster[1]) >= min_sample:
            centermost_points.append(get_centermost_point(cluster[1]))
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
    # pull row from original data set where lat/lon match the lat/lon of each row of representative points
    # that way we get the full details from the original dataframe
    rs = rep_points.apply(lambda row: df[(df['LATITUDE']==row['lat']) & 
                                             (df['LONGITUDE']==row['lon'])].iloc[0], axis=1)
    return rs

In [9]:
def plotHotSpots(orgdf, rset):
    # plot the final reduced set of coordinate points vs the original full set
    fig, ax = plt.subplots(figsize=[17.5, 17])
    df_scatter1 = ax.scatter(orgdf['LONGITUDE'], orgdf['LATITUDE'], c='k', alpha=0.9, s=3)
    rs_scatter1 = ax.scatter(rset['LONGITUDE'], rset['LATITUDE'], c='#FF8C00', edgecolor='None', alpha=0.7, s=120)
    ax.set_title('Full data set vs DBSCAN reduced set')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_xlim(-74.06, -73.77)
    ax.set_ylim(40.61, 40.91)
    ax.legend([df_scatter1, rs_scatter1], ['Full set', 'Reduced set'], loc='upper right')
    plt.show()

In [10]:
def buildFoliumMap(rs):
    NY_COORDINATES = (40.61, -74.06)
    # create empty map zoomed in on NYC
    map = folium.Map(location=NY_COORDINATES, zoom_start=11)
    
    for each in rs.iterrows():
        folium.CircleMarker(
            location = [each[1]['LATITUDE'],each[1]['LONGITUDE']],
            radius = 12, popup= str(each[1]['BOROUGH']) + ": " + str(each[1]["ON.STREET.NAME"]) + "(zip:" + str(each[1]["ZIP.CODE"]) + ")",
        fill=True, color='#3186cc', fill_color='#FF8C00', fill_opacity=0.7).add_to(map)
    return map

In [11]:
def plotHotSpots_Folium(rs):
    display(buildFoliumMap(rs))

In [12]:
kms_per_radian = 6371.0088

In [81]:
def prepareHotSpotsForFullData(df_fulldata):
    rs = pd.DataFrame()
    coords = getcoords(df_fulldata)
    for min_sample in range(10, 110, 10):
        for i in range(0, 11, 1):
            if i != 0:
                tmp = i / 10.0
                epsilon = tmp / kms_per_radian
                clusters = applyDBSCAN(coords, epsilon, min_sample)
                hsdata = gethotspots(clusters, df_fulldata, min_sample)
                hsdata['EPSILON'] = tmp
                hsdata['SAMPLES'] = min_sample
                rs = rs.append(hsdata)
    return rs

In [98]:
#Clean and get right dataframe
def cleandf(df,vz):
    df.drop(df.iloc[:, 0:2], inplace=True, axis=1)
    df['VZ'] = vz
    filter = (df["LATITUDE"].notnull()) & (df["LONGITUDE"].notnull())
    df = df[filter]  
    df['YEAR'], df['MONTH'],df['DAY_OF_WEEK']= df['DATE'].apply(lambda x: x.year), df['DATE'].apply(lambda x: x.month), df['DATE'].apply(lambda x: x.weekday())
    df['NBR_VZ_INITIATIVES'] = df['SLOWZONE'] + df['SPEED_HUMP'] + df['SIGNAL_TIMING'] + df['BIKE_PRIORITY'] + df['ENHANCED_CROSSING'] + df['LEAD_PEDESTRIAN_INTERVAL'] + df['LEFT_TURN_TRAFFIC_CALMING'] + df['NEIGHBORHOOD_SLOW_ZONE'] + df['SAFE_STREETS_FOR_SENIORS'] + df['SIP_INTERSECTIONS']+ df['SAFE_CORRIDORS'] + df['PRIORITY_CORRIDORS'] + df['PRIORITY_INTERSECTIONS'] + df['PRIORITY_ZONES']
    return df

In [ ]:
#df.to_csv('pedestrian-incidents-fulldata.csv', encoding='utf-8', index=False)

## Load the pedestrian data before vision zero initiative

In [69]:
dfped_nvz = pd.read_csv('LatestData/EDA/pedestrian_nvz.csv', parse_dates=[2])

In [70]:
dfped_nvz = cleandf(dfped_nvz, 0)
dfped_nvz.shape

C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(24496, 47)

In [71]:
dfped_nvz.head()

,DATE,TIME,BOROUGH,ZIP.CODE,LATITUDE,LONGITUDE,LOCATION,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME,...,SAFE_STREETS_FOR_SENIORS,SIP_INTERSECTIONS,SAFE_CORRIDORS,PRIORITY_CORRIDORS,PRIORITY_INTERSECTIONS,PRIORITY_ZONES,VZ,YEAR,MONTH,DAY_OF_WEEK
1,2014-04-08,11:45,BROOKLYN,11211.0,40.706479,-73.963744,"(40.7064792, -73.9637444)",BEDFORD AVENUE,CLYMER STREET,NaN,...,0,0,0,0,0,0,0,2014,4,1
2,2014-04-08,12:00,MANHATTAN,10019.0,40.768382,-73.992805,"(40.7683816, -73.992805)",11 AVENUE,WEST 54 STREET,NaN,...,0,0,0,0,0,0,0,2014,4,1
3,2014-04-08,12:27,MANHATTAN,10075.0,40.773336,-73.955054,"(40.7733356, -73.9550538)",EAST 79 STREET,2 AVENUE,NaN,...,1,0,1,1,1,1,0,2014,4,1
4,2014-04-08,12:30,BROOKLYN,11213.0,40.669402,-73.942197,"(40.6694023, -73.9421971)",KINGSTON AVENUE,EASTERN PARKWAY,NaN,...,0,0,1,1,1,1,0,2014,4,1
5,2014-04-08,13:40,MANHATTAN,10017.0,40.751453,-73.978153,"(40.751453, -73.9781533)",EAST 41 STREET,PARK AVENUE,NaN,...,0,0,1,0,0,1,0,2014,4,1


# Load the pedestrian data after vision zero initiative

In [74]:
dfped_vz = pd.read_csv('LatestData/EDA/pedestrian_vz.csv', parse_dates=[2])

In [75]:
dfped_vz = cleandf(dfped_vz, 1)
dfped_vz.shape

C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(50969, 47)

In [76]:
dfped_vz.head()

,DATE,TIME,BOROUGH,ZIP.CODE,LATITUDE,LONGITUDE,LOCATION,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME,...,SAFE_STREETS_FOR_SENIORS,SIP_INTERSECTIONS,SAFE_CORRIDORS,PRIORITY_CORRIDORS,PRIORITY_INTERSECTIONS,PRIORITY_ZONES,VZ,YEAR,MONTH,DAY_OF_WEEK
0,2018-04-28,0:00,NaN,NaN,40.685290,-73.976204,"(40.68529, -73.976204)",HANSON PLACE,NaN,NaN,...,0,0,0,0,0,1,1,2018,4,5
1,2018-04-28,10:30,BRONX,10462.0,40.854122,-73.867714,"(40.854122, -73.867714)",NaN,NaN,2126 WHITE PLAINS ROAD,...,0,0,1,1,0,0,1,2018,4,5
2,2018-04-28,10:30,NaN,NaN,40.707070,-73.817190,"(40.70707, -73.81719)",QUEENS BOULEVARD,NaN,NaN,...,0,0,0,1,0,1,1,2018,4,5
3,2018-04-28,12:00,BROOKLYN,11203.0,40.656063,-73.939570,"(40.656063, -73.93957)",CLARKSON AVENUE,ALBANY AVENUE,NaN,...,0,0,0,0,0,0,1,2018,4,5
4,2018-04-28,12:35,NaN,NaN,40.854626,-73.895160,"(40.854626, -73.89516)",PARK AVENUE,NaN,NaN,...,0,0,0,0,0,1,1,2018,4,5


# Combine both into one pedestrian data file

In [77]:
dfped_full = dfped_nvz.append(dfped_vz)

In [78]:
dfped_full.shape

(75465, 47)

In [92]:
#Write into a csv file
dfped_full.to_csv('pedestrian-incidents-fulldata-latest.csv', encoding='utf-8', index=False)

# Find HotSpots and Persist

In [82]:
hotspots = prepareHotSpotsForFullData(dfped_full)

1372
Clustered 75,465 points down to 1,372 clusters, for 98.2% compression in 3.06 seconds
250
Clustered 75,465 points down to 250 clusters, for 99.7% compression in 3.21 seconds
78
Clustered 75,465 points down to 78 clusters, for 99.9% compression in 3.85 seconds
41
Clustered 75,465 points down to 41 clusters, for 99.9% compression in 4.81 seconds
23
Clustered 75,465 points down to 23 clusters, for 100.0% compression in 5.20 seconds
15
Clustered 75,465 points down to 15 clusters, for 100.0% compression in 6.09 seconds
14
Clustered 75,465 points down to 14 clusters, for 100.0% compression in 6.50 seconds
12
Clustered 75,465 points down to 12 clusters, for 100.0% compression in 7.55 seconds
8
Clustered 75,465 points down to 8 clusters, for 100.0% compression in 8.26 seconds
7
Clustered 75,465 points down to 7 clusters, for 100.0% compression in 8.84 seconds
693
Clustered 75,465 points down to 693 clusters, for 99.1% compression in 2.75 seconds
245
Clustered 75,465 points down to 245 clu

14
Clustered 75,465 points down to 14 clusters, for 100.0% compression in 5.69 seconds
14
Clustered 75,465 points down to 14 clusters, for 100.0% compression in 6.79 seconds
9
Clustered 75,465 points down to 9 clusters, for 100.0% compression in 7.28 seconds
8
Clustered 75,465 points down to 8 clusters, for 100.0% compression in 7.83 seconds
6
Clustered 75,465 points down to 6 clusters, for 100.0% compression in 9.00 seconds


In [83]:
hotspots.shape

(5452, 49)

In [94]:
hotspots.to_csv('pedestrian-hotspots-fulldata-latest.csv', encoding='utf-8', index=False)

# Load 3 vehicle incident data and do the same hotspot findings

In [105]:
df3veh_nvz = pd.read_csv('LatestData/EDA/threevehicles_nvz.csv', parse_dates=[2])

In [106]:
df3veh_nvz = cleandf(df3veh_nvz, 0)

C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [107]:
df3veh_nvz.head()

,DATE,TIME,BOROUGH,ZIP.CODE,LATITUDE,LONGITUDE,LOCATION,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME,...,SIP_INTERSECTIONS,SAFE_CORRIDORS,PRIORITY_CORRIDORS,PRIORITY_INTERSECTIONS,PRIORITY_ZONES,VZ,YEAR,MONTH,DAY_OF_WEEK,NBR_VZ_INITIATIVES
2,2014-04-08,7:20,NaN,NaN,40.585165,-73.956406,"(40.5851653, -73.9564056)",NaN,NaN,NaN,...,0,0,0,0,0,0,2014,4,1,1
3,2014-04-08,10:13,BRONX,10451.0,40.811723,-73.928126,"(40.8117235, -73.9281256)",EAST 138 STREET,RIDER AVENUE,NaN,...,0,0,1,0,1,0,2014,4,1,4
5,2014-04-08,10:15,BROOKLYN,11234.0,40.618022,-73.914214,"(40.6180217, -73.9142142)",AVENUE T,EAST 64 STREET,NaN,...,0,0,0,0,0,0,2014,4,1,0
6,2014-04-08,12:17,NaN,NaN,40.711541,-73.836232,"(40.7115409, -73.8362317)",NaN,NaN,NaN,...,0,0,0,0,0,0,2014,4,1,0
7,2014-04-08,12:20,NaN,NaN,40.806346,-73.933172,"(40.8063462, -73.9331715)",NaN,NaN,NaN,...,0,0,0,0,0,0,2014,4,1,0


In [108]:
df3veh_vz = pd.read_csv('LatestData/EDA/threevehicles_vz.csv', parse_dates=[2])
df3veh_vz = cleandf(df3veh_vz, 1)
df3veh_vz.head()

C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\p_sum\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,DATE,TIME,BOROUGH,ZIP.CODE,LATITUDE,LONGITUDE,LOCATION,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME,...,SIP_INTERSECTIONS,SAFE_CORRIDORS,PRIORITY_CORRIDORS,PRIORITY_INTERSECTIONS,PRIORITY_ZONES,VZ,YEAR,MONTH,DAY_OF_WEEK,NBR_VZ_INITIATIVES
0,2018-04-28,0:20,NaN,NaN,40.680977,-73.911385,"(40.680977, -73.911385)",SUMPTER STREET,NaN,NaN,...,0,0,1,0,1,1,2018,4,5,2
1,2018-04-28,10:30,NaN,NaN,40.707070,-73.817190,"(40.70707, -73.81719)",QUEENS BOULEVARD,NaN,NaN,...,0,0,1,0,1,1,2018,4,5,2
2,2018-04-28,12:47,MANHATTAN,10025.0,40.800186,-73.959145,"(40.800186, -73.959145)",NaN,NaN,12 WEST 109 STREET,...,0,0,0,0,0,1,2018,4,5,1
3,2018-04-28,13:15,NaN,NaN,40.798256,-73.827440,"(40.798256, -73.82744)",BRONX WHITESTONE BRIDGE,NaN,NaN,...,0,0,0,0,0,1,2018,4,5,0
4,2018-04-28,13:30,QUEENS,11365.0,40.730550,-73.800530,"(40.73055, -73.80053)",71 AVENUE,169 STREET,NaN,...,0,0,0,0,0,1,2018,4,5,0


In [109]:
df3veh_full = df3veh_nvz.append(df3veh_vz)

In [110]:
df3veh_full.shape

(64620, 48)

In [111]:
#Write into a csv file
df3veh_full.to_csv('threevehicle-incidents-fulldata-latest.csv', encoding='utf-8', index=False)

In [112]:
hotspots = prepareHotSpotsForFullData(df3veh_full)

1556
Clustered 64,620 points down to 1,556 clusters, for 97.6% compression in 3.21 seconds
311
Clustered 64,620 points down to 311 clusters, for 99.5% compression in 2.99 seconds
87
Clustered 64,620 points down to 87 clusters, for 99.9% compression in 3.49 seconds
32
Clustered 64,620 points down to 32 clusters, for 100.0% compression in 4.38 seconds
15
Clustered 64,620 points down to 15 clusters, for 100.0% compression in 4.50 seconds
12
Clustered 64,620 points down to 12 clusters, for 100.0% compression in 4.79 seconds
10
Clustered 64,620 points down to 10 clusters, for 100.0% compression in 5.18 seconds
9
Clustered 64,620 points down to 9 clusters, for 100.0% compression in 5.95 seconds
9
Clustered 64,620 points down to 9 clusters, for 100.0% compression in 7.03 seconds
9
Clustered 64,620 points down to 9 clusters, for 100.0% compression in 6.55 seconds
372
Clustered 64,620 points down to 372 clusters, for 99.4% compression in 2.60 seconds
426
Clustered 64,620 points down to 426 clus

19
Clustered 64,620 points down to 19 clusters, for 100.0% compression in 5.79 seconds
9
Clustered 64,620 points down to 9 clusters, for 100.0% compression in 6.41 seconds
7
Clustered 64,620 points down to 7 clusters, for 100.0% compression in 6.81 seconds
6
Clustered 64,620 points down to 6 clusters, for 100.0% compression in 7.60 seconds
5
Clustered 64,620 points down to 5 clusters, for 100.0% compression in 10.15 seconds


In [113]:
hotspots.shape

(5650, 50)

In [114]:
hotspots.to_csv('threevehicle-hotspots-fulldata-latest.csv', encoding='utf-8', index=False)